This Jupyter notebook is to learn an enire process of deep learning applications: i.e., interactive data collection, training, and testing. This material is largely based on the NVIDIA Deep Learning Institute (DLI) course: Getting Started with AI on Jetson Nano. The current implementation is based on PyTorch. You can also find TensorFlow code in the following [link](https://colab.research.google.com/drive/1eym2fLQNkNYl_HCtsLspFdSYD_n_xjDZ). we found that PyTorch requires less memory footprint when compared with TensorFlow. When multiple threads are running simultaneously, TensorFlow becomes so slow--it seems like there is a memory issue.

 Alternatively, you can install [TensorFlow Lite](https://www.tensorflow.org/lite/guide/get_started), which provides all the tools you need to convert and run TensorFlow models on mobile, embedded, and IoT devices. It support both mobile platforms (Android and iOS) as well as other linux platforms. You should be able to install TensorFlow Lite to Jetson Nano; e.g., please see these articles: lite @ [medium](https://medium.com/@yanweiliu/tflite-on-jetson-nano-c480fdf9ac2) or [stackoverflow](https://stackoverflow.com/questions/60871843/tensorflow-lite-on-nvidia-jetson). There is an interesting article that compares performance of different platforms: ["Google Coral Edge TPU vs NVIDIA Jetson Nano: A quick deep dive into EdgeAI performance".](https://blog.usejournal.com/google-coral-edge-tpu-vs-nvidia-jetson-nano-a-quick-deep-dive-into-edgeai-performance-bc7860b8d87a?gi=2b6d1b4feb13)


#  Data Collection


The output of our model is a set of four probabilities:
- **p(Left)** - a probability of turning left (spinning counterclockwise)
- **p(right)** - a probability of turning right (spinning clockwise)
- **p(blocked)** - a probability of the path being blocked
- **p(free)** - a probability of no obstacles in front of the robot (so it is safe to move forward)

The following method is used collect the data:  

First, you'll manually place the robot in scenarios where its "safety bubble" is violated, and label these scenarios ``blocked``.  You save a snapshot of what the robot sees along with this label.

Second, you'll manually place the robot in scenarios where it's safe to move forward a bit, and label these scenarios ``free``.  Likewise, we save a snapshot along with this label.

Thrird, you'll manually place the robot in scenarios where spinning to the left (counterclockwise) would be the optimal move and label these scenarios ``left``. Likewise, you save a snapshot along with this label. Try to vary the angle of the desired rotation - place the robot in scenarios where this angle is larger or smaller.

Finally, you'll manually place the robot in scenarios where turning right (clockwise) would be the optimal move and label these scenarios ``right``. Likewise, you save a snapshot along with this label. Try to vary the angle of the desired rotation - place the robot in scenarios where this angle is larger or smaller. 

Please refer to the following pictures!  
 


<img src="https://ifh.cc/g/8N3mfC.jpg" width=200>



Once you have collected 100-200 images for each of four classes, do either one of the following: 
1. **Upload this data to CoLab for training and download the trained model at your Jetbot**
2. Alternatively, train a model locally using JetBot's GPU  

But we recommand the first option due to speed reasons!! (it's much faster to train a model at Colab). 

### Camera

First, let's initialize and display

> This block sets the size of the images and starts the camera. If your camera is already active in this notebook or in another notebook, first shut down the kernel in the active notebook before running this code cell. Make sure that the correct camera type is selected for execution (CSI or USB). This cell may take several seconds to execute.

In [0]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg


camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  #   width and height do not necessarily have to match the camera

# Traitlets is a framework that lets Python classes have attributes with type checking, dynamically calculated default values, and ‘on change’ callbacks. 
# https://traitlets.readthedocs.io/_/downloads/en/stable/pdf/ 
# dlink: Link the trait of a source object with traits of target objects.
# https://traitlets.readthedocs.io/en/stable/utils.html
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

#display(image)

### Task

This class is for constructing a dataset, and the image from the JetBot camerais configured as a dataset. 



*   The ```__getitem__``` method loads your image files and convert them from an jpg file to an array so that you can learn with PyTorch.
*   The ```_refresh``` method annotates the path and category at the image you saved.
*   The ```save_entry``` method  is a function to save the image received from the camera to a prefined path. Your image files' name determined by ```uuid```, ``uuid`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.
*   The ```get_count``` method  is a function that stores how many images are stored in your path.



In [0]:
import torch
import torch.utils.data
import glob
import PIL.Image
import subprocess
import cv2
import os
import uuid
import subprocess


class ImageClassificationDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, categories, transform=None):
        self.categories = categories
        self.directory = directory
        self.transform = transform
        self._refresh()
    
    
    def __len__(self):
        return len(self.annotations)
    
    
    def __getitem__(self, idx):
        ann = self.annotations[idx]
        image = cv2.imread(ann['image_path'], cv2.IMREAD_COLOR)
        image = PIL.Image.fromarray(image)
        if self.transform is not None:
            image = self.transform(image)
        return image, ann['category_index']
    
    
    def _refresh(self):
        self.annotations = []
        for category in self.categories:
            category_index = self.categories.index(category)
            for image_path in glob.glob(os.path.join(self.directory, category, '*.jpg')):
                self.annotations += [{
                    'image_path': image_path,
                    'category_index': category_index,
                    'category': category
                }]
    
    def save_entry(self, image, category):
        """Saves an image in BGR8 format to dataset for category"""
        if category not in self.categories:
            raise KeyError('There is no category named %s in this dataset.' % category)
            
        filename = str(uuid.uuid1()) + '.jpg'
        category_directory = os.path.join(self.directory, category)
        
        if not os.path.exists(category_directory):
            subprocess.call(['mkdir', '-p', category_directory])
            
        image_path = os.path.join(category_directory, filename)
        cv2.imwrite(image_path, image)
        self._refresh()
        return image_path
    
    def get_count(self, category):
        i = 0
        for a in self.annotations:
            if a['category'] == category:
                i += 1
        return i

You can organize your dataset by assigning tasks and categories

In [0]:
# You can chage your task and categories by controlling follwing variables 
TASK = 'driving'
CATEGORIES = ['free', 'left', 'right', 'stop']
DATASETS = ['A', 'B']

The images saved by JetBot are in jpg format. 
In order for your robot to learn an image, it is necessary to change the jpg file to a numeric value between [0-225] and convert it into a form  that PyTorch can learn.
So when you load your image file, it need to transform properly. (from jpg to torch)


Fortunately, PyTorch provides a transform module that can easily transform an image. There are various image transform methods(e.g., color, size), They can be chained together using ```Compose```


*   ```ColorJitter``` provide function change the brightness, contrast and saturation of an image. 
*   ```Resize``` provide your saved image to definded size
*   ```ToTensor``` provide send your data CPU to GPU
*   ```Normalize``` provide normalize a tensor image with mean and standard deviation

You can check detail here: https://pytorch.org/docs/stable/torchvision/transforms.html

In [0]:
import torchvision.transforms as transforms

# camera = Camera.instance(width=224, height=224) 

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2), # ColorJitter(brightness, contrast, saturation, hue)
    transforms.Resize((224, 224)), #  Resize((width, height))
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #torchvision.transforms.Normalize(mean, std) 
    # Given mean: (M1,...,Mn) and std: (S1,..,Sn) for n channels, this transform will normalize each channel of the input
    # In this case, the number of channel is 3 (RGB)
])

# After setting your task and data transformation method, create an instance of the dataset using ImageClassificationDataset.
datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS)
    
print("{} task with {} categories defined".format(TASK, CATEGORIES))

driving task with ['free', 'left', 'right', 'blocked'] categories defined


### Data collection

You'll collect images for your categories with your camera using an iPython widget. This cell sets up the collection mechanism to count your images and produce the user  

In [0]:
import ipywidgets
# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview (camera widget: Provides images observed by the camera in real time)
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets  (Widget for organizing datasets)
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')

# manually update counts at initialization 
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    dataset.save_entry(camera.value, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), dataset_widget, category_widget, count_widget, save_widget
])


display(data_collection_widget)
print("data_collection_widget created")

If you refresh the Jupyter file browser on the left, you should now see those directories appearing.  Next, let's create and display some buttons that you'll use to save snapshots for each class label.  You'll also add some text boxes that will display how many images of each category that we've collected so far. This is useful because we want to make sure you collect about the same number of images for each class (``free``, ``left``, ``right`` or ``blocked``).  It also helps to know how many images we've collected overall.


Now go ahead and collect some data 

1. Place the robot in a scenario where it's supposed to turn right and press ``add right``
2. Place the robot in a scenario where it's supposed to turn left and press ``add left``
3. Place the robot in a scenario where it's free and press ``add free``
3. Place the robot in a scenario where it's blocked and press ``add blocked``
5. Repeat 1, 2, 3, 4


Here are some tips for labeling data

1. Try different orientations (e.g. sharp right vs slight right, closer to the cup or further away from it, etc.) 
2. Try different lighting
3. Try different textured floors / objects;  patterned, smooth, glass, etc.

Ultimately, the more data we have of scenarios the robot will encounter in the real world, the better our collision avoidance behavior will be.  It's important
to get *varied* data (as described by the above tips) and not just a lot of data, but you'll probably need at least 100 images of each class (that's not a science, just a helpful tip here).

## Next

Once you've collected enough data, you'll need to copy that data to our GPU desktop or cloud machine for training.  First, we can call the following *terminal* command to compress
your dataset folder into a single *zip* file.


In [0]:
!zip -r -q dataset.zip driving_A   # set your dataset folder

You should see a file named dataset.zip in the Jupyter Lab file browser.  You should download the zip file using the Jupyter Lab file browser by right clicking and selecting Download.

After that, you upload your zip files in google drive. 
3_Triain_model.ipynb will perform at Colab using a GPU. 
